In [207]:
import pandas as pd
import numpy as np

In [208]:
#importing data (pre1790)
pre1790 = pd.read_csv('prepost_data_aggregations/statewise_debt_aggregation_Pre1790.csv')
pre1790.columns = ['state', 'Loan Office Certificates Specie Value','State Liquidated Debt Dollars','Pierce Certificates Value']

In [220]:
#importing data (post1790)
post1790 = pd.read_csv('prepost_data_aggregations/statewise_debt_aggregation_Post1790.csv', index_col = 0)
post1790_ds = pd.read_csv('prepost_data_aggregations/statewise_debt_aggregation_Post1790_debtstate.csv', index_col = 0)
post1790_rep = pd.read_csv('prepost_data_aggregations/statewise_debt_aggregation_Post1790_rep.csv', index_col = 0)

In [210]:
#pre 1790 and post1790 asset totals
pre1790cols = ['Loan Office Certificates Specie Value', 'State Liquidated Debt Dollars', 'Pierce Certificates Value']
post1790cols = ['ASD Total', 'CD Total']
pre1790['pre1790 total'] = pre1790[pre1790cols].sum(axis = 1)
post1790['post1790 total'] = post1790[post1790cols].sum(axis = 1)
post1790_ds['post1790 total'] = post1790_ds[post1790cols].sum(axis = 1)
post1790_rep['post1790 total'] = post1790_rep[post1790cols].sum(axis = 1)
pre1790_tot = pre1790['pre1790 total'].sum()
post1790_tot = post1790['post1790 total'].sum()
print("pre1790: {:.2f}, post1790: {:.2f}, ratio: {:.2f}".format(pre1790_tot, post1790_tot, post1790_tot/pre1790_tot))

pre1790: 26860611.73, post1790: 14476118.95, ratio: 0.54


In [211]:
def calculateMetrics(df):
    #aggregates
    df = df.fillna(0)
    df['pre1790 pct'] = df['pre1790 total']/pre1790_tot * 100
    df['post1790 pct'] = df['post1790 total']/post1790_tot * 100
    df['total asset dif'] = df['post1790 total'] - df['pre1790 total']
    df['total asset dif (adj)'] = df['post1790 total']*pre1790_tot/post1790_tot - all_debts_ds['pre1790 total']
    df['total asset dif (pct)'] = (df['post1790 pct'] - df['pre1790 pct'])
    
    df['Loan Office pct'] =  df['Loan Office Certificates Specie Value']/df['Loan Office Certificates Specie Value'].sum()
    df['State Liquidated pct'] =  df['State Liquidated Debt Dollars']/df['State Liquidated Debt Dollars'].sum()
    df['Pierce pct'] =  df['Pierce Certificates Value']/df['Pierce Certificates Value'].sum()
    df['CD pct'] =  df['CD Total']/df[ 'CD Total'].sum()
    df['ASD pct'] =  df['ASD Total']/df['ASD Total'].sum()
            
    return df.round(2)

In [212]:
#default

In [213]:
all_debts = pd.merge(pre1790[['state', 'pre1790 total']], post1790[['state', 'post1790 total']], 
                     on = 'state', how = 'outer').round(2)
#all_debts

In [214]:
post1790_ds.columns = ['state', 'ASD Total', 'CD Total', 'post1790 total']
all_debts_ds = pd.merge(pre1790, post1790_ds, 
                        on = 'state', how = 'outer').round(2)
all_debts_ds = calculateMetrics(all_debts_ds)
all_debts_ds

,state,Loan Office Certificates Specie Value,State Liquidated Debt Dollars,Pierce Certificates Value,pre1790 total,ASD Total,CD Total,post1790 total,pre1790 pct,post1790 pct,total asset dif,total asset dif (adj),total asset dif (pct),Loan Office pct,State Liquidated pct,Pierce pct,CD pct,ASD pct
0,PA,3778570.27,2257929.43,1147498.52,7183998.22,0.00,2952380.76,2952380.76,26.75,20.39,-4231617.46,-1705820.43,-6.35,0.38,0.26,0.14,0.26,0.00
1,MA,2387481.08,457823.49,2217598.90,5062903.47,0.00,0.00,0.00,18.85,0.00,-5062903.47,-5062903.47,-18.85,0.24,0.05,0.27,0.00,0.00
2,CT,1275499.48,135432.22,1137988.73,2548920.43,366794.04,1389315.78,1756109.82,9.49,12.13,-792810.61,709562.33,2.64,0.13,0.02,0.14,0.12,0.11
3,NY,922005.66,3649822.51,774934.83,5346763.00,1408932.24,1926464.36,3335396.60,19.91,23.04,-2011366.40,842105.26,3.14,0.09,0.42,0.09,0.17,0.44
4,NJ,651204.36,1768007.24,513348.62,2932560.22,0.00,0.00,0.00,10.92,0.00,-2932560.22,-2932560.22,-10.92,0.06,0.21,0.06,0.00,0.00
5,MD,404704.96,0.00,550014.80,954719.76,143823.61,1447729.90,1591553.51,3.55,10.99,636833.75,1998426.80,7.44,0.04,0.00,0.07,0.13,0.04
6,NH,300171.07,13826.44,380285.90,694283.41,0.00,307782.89,307782.89,2.58,2.13,-386500.52,-123188.58,-0.46,0.03,0.00,0.05,0.03,0.00
7,VA,235249.60,0.00,653914.72,889164.32,264976.95,859264.17,1124241.12,3.31,7.77,235076.80,1196878.52,4.46,0.02,0.00,0.08,0.08,0.08
8,DE,66387.12,136305.94,130868.22,333561.28,0.00,0.00,0.00,1.24,0.00,-333561.28,-333561.28,-1.24,0.01,0.02,0.02,0.00,0.00
9,RI,0.00,183000.46,81210.76,264211.22,64363.22,814182.09,878545.31,0.98,6.07,614334.09,1365940.10,5.09,0.00,0.02,0.01,0.07,0.02


In [215]:
all_debts_rep = pd.merge(pre1790, post1790_rep, 
                        on = 'state', how = 'outer').round(2)
all_debts_rep.loc[all_debts_rep[all_debts_rep['state'] == 'NY loan office'].index, 'state'] = 'NY'
all_debts_rep.groupby('state')[['pre1790 total','post1790 total']].sum().reset_index()
all_debts_rep = calculateMetrics(all_debts_rep)
all_debts_rep

,state,Loan Office Certificates Specie Value,State Liquidated Debt Dollars,Pierce Certificates Value,pre1790 total,ASD Total,CD Total,post1790 total,pre1790 pct,post1790 pct,total asset dif,total asset dif (adj),total asset dif (pct),Loan Office pct,State Liquidated pct,Pierce pct,CD pct,ASD pct
0,PA,3778570.27,2257929.43,1147498.52,7183998.22,129119.33,2843338.75,2972458.08,26.75,20.53,-4211540.14,-1668566.72,-6.21,0.38,0.26,0.14,0.25,0.04
1,MA,2387481.08,457823.49,2217598.90,5062903.47,18523.71,135257.04,153780.75,18.85,1.06,-4909122.72,-4777561.45,-17.79,0.24,0.05,0.27,0.01,0.01
2,CT,1275499.48,135432.22,1137988.73,2548920.43,282429.55,1346300.72,1628730.27,9.49,11.25,-920190.16,473208.05,1.76,0.13,0.02,0.14,0.12,0.09
3,NY,922005.66,3649822.51,774934.83,5346763.00,353945.33,25985.37,379930.70,19.91,2.62,-4966832.30,-4641797.05,-17.28,0.09,0.42,0.09,0.00,0.11
4,NJ,651204.36,1768007.24,513348.62,2932560.22,1607.80,43128.01,44735.81,10.92,0.31,-2887824.41,-2849552.39,-10.61,0.06,0.21,0.06,0.00,0.00
5,MD,404704.96,0.00,550014.80,954719.76,141713.62,1436570.68,1578284.30,3.55,10.90,623564.54,1973805.62,7.35,0.04,0.00,0.07,0.13,0.04
6,NH,300171.07,13826.44,380285.90,694283.41,544.07,272954.78,273498.85,2.58,1.89,-420784.56,-186803.02,-0.70,0.03,0.00,0.05,0.02,0.00
7,VA,235249.60,0.00,653914.72,889164.32,275087.60,886516.08,1161603.68,3.31,8.02,272439.36,1266205.19,4.71,0.02,0.00,0.08,0.08,0.09
8,DE,66387.12,136305.94,130868.22,333561.28,0.00,12781.59,12781.59,1.24,0.09,-320779.69,-309844.89,-1.15,0.01,0.02,0.02,0.00,0.00
9,RI,0.00,183000.46,81210.76,264211.22,62807.17,766286.96,829094.13,0.98,5.73,564882.91,1274182.85,4.74,0.00,0.02,0.01,0.07,0.02


In [216]:
all_debts_ds.to_csv('statewise_all_assets_ds.csv')

In [217]:
all_debts_rep.to_csv('statewise_all_assets_rep.csv')